# Part 1: Introduction to Decision Trees

## The decision tree algorithm is a supervised learning algorithm, first construct the tree with historical data, then use it to predict an outcome.

In [1]:
import pandas as pd
pd.options.display.max_columns = 100
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
## Incividual income data, from 1994 census.

## Set index_col to False to avoid pandas thinking that the first column is row indexes (it's age)
income = pd.read_csv('income.csv', index_col=False)
income_original = income.copy()
income.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,high_income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [3]:
income['high_income'].value_counts()

 <=50K    24720
 >50K      7841
Name: high_income, dtype: int64

## Category column, workclass, sex, convert them to numerical values.

## Convert the columns to a categorical type. Pandas will display the labels as strings, but inernally store them as numbers so we can do computations with them.

In [4]:
col = pd.Categorical(income['workclass'])
income['workclass'] = col.codes
income['workclass'].head(5)

0    7
1    6
2    4
3    4
4    4
Name: workclass, dtype: int8

In [5]:
cols = ['education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'native_country',
       'high_income']
for col in cols:
    col_cat = pd.Categorical(income[col])
    income[col] = col_cat.codes

In [6]:
income.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,high_income
0,39,7,77516,9,13,4,1,1,4,1,2174,0,40,39,0
1,50,6,83311,9,13,2,4,0,4,1,0,0,13,39,0
2,38,4,215646,11,9,0,6,1,4,1,0,0,40,39,0
3,53,4,234721,1,7,2,6,0,2,1,0,0,40,39,0
4,28,4,338409,9,13,2,10,5,2,0,0,0,40,5,0


In [7]:
income.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   age             32561 non-null  int64
 1   workclass       32561 non-null  int8 
 2   fnlwgt          32561 non-null  int64
 3   education       32561 non-null  int8 
 4   education_num   32561 non-null  int64
 5   marital_status  32561 non-null  int8 
 6   occupation      32561 non-null  int8 
 7   relationship    32561 non-null  int8 
 8   race            32561 non-null  int8 
 9   sex             32561 non-null  int8 
 10  capital_gain    32561 non-null  int64
 11  capital_loss    32561 non-null  int64
 12  hours_per_week  32561 non-null  int64
 13  native_country  32561 non-null  int8 
 14  high_income     32561 non-null  int8 
dtypes: int64(6), int8(9)
memory usage: 1.8 MB


## Split income into two parts based on the value of workclass column

In [8]:
private_incomes = income[income['workclass'] == 4]
public_incomes = income[income['workclass'] != 4]

In [9]:
print(private_incomes.shape)
print(public_incomes.shape)

(22696, 15)
(9865, 15)


## Think of a decision tree as a flow of rows of data. When we make a split, some rows will go to the right, and some will go to the left. As we build the tree deeper and deeper, each node will "receive" fewer and fewer rows.

## The nodes at the bottom of the tree, where we decide to stop splitting, are called terminal nodes, or leaves. To ensure that we can make a prediction on future data, all rows in each leaf must have only one value for the target column.

## Try to predict the high_income column.

## In order to be able to make prediction for high_income, all of the rows in a leaf should only have a single value for high_income. Leaves can't have both 0 and 1 values in the high_income column.

***
* Information theory is based on probability and statistics, and deals with the transmission, processing, utilizaton and extraction of information. A bit of information, one bit of information is one unit of information.
***

## When we split, we'll try to separate as many 0s from 1s in the high_income column as we can. In order to do this, we need a metric for how 'together' the difference values in the high_income columns are, commonly use a metric called entropy for this purpose. Entropy refers to disorder. The more "mixed together" 1 s and 0 s are, the higher the entropy.

## The formula for entropy looks like this: $-\sum_{i=1}^{c}P(x_i)log_bP(x_i)$

age    high_income
***
25     1
***
50     1
***

30     0
***
50     0
***
80     1

In [10]:
# For the above data

import math
entropy = -(2/5 * math.log(2/5, 2) + 3/5 * math.log(3/5, 2))
print(entropy)

0.9709505944546686


We get less than one "bit" of information -- only .97 -- because there are slightly more 1 s in the sample data than 0 s. This means that if we were predicting a new value, we could guess that the answer is 1 and be right more often than wrong (because there's  a .6 probability of the answer being 1). Due to this prior knowledge, we gain less than a full "bit" of information when we observe a new value.!!!

In [11]:
print(income.high_income.value_counts())
print(income.shape[0])

0    24720
1     7841
Name: high_income, dtype: int64
32561


In [12]:
total = income.shape[0]
number_0 = income[income['high_income'] == 0].shape[0]
number_1 = total - number_0
income_entropy = -(number_0 / total * math.log(number_0 / total, 2) + number_1 / total * math.log(number_1/total, 2))
income_entropy

0.7963839552022132

In [13]:
income.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,high_income
0,39,7,77516,9,13,4,1,1,4,1,2174,0,40,39,0
1,50,6,83311,9,13,2,4,0,4,1,0,0,13,39,0
2,38,4,215646,11,9,0,6,1,4,1,0,0,40,39,0
3,53,4,234721,1,7,2,6,0,2,1,0,0,40,39,0
4,28,4,338409,9,13,2,10,5,2,0,0,0,40,5,0


In [14]:
income_original.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,high_income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [15]:
income[income['sex'] == 1].shape

(21790, 15)

## Which split will reduce entropy the most from information gain.
## $IG(T,A) = Entropy(T) - \sum_{v\in A}\frac{|T_n|}{|T|}\cdot Entropy{(T_n)}$
* IG - information gain (IG) for a given target variable(T), as well as a given variable we want to split on (A).

## To compute IG(T,A), we first calculate the entropy for T. Then,for each unique value *v* in the variable A, we compute the number of rows in which A takes on the value v, and divide it by the total number of rows. Next, we multiply the results by the entropy of the rows where A is v. We add all of these subset entropies together, then substract from the overall entropy to get information gain.

## Alternate explanation: We're finding the entropy of each set post-split,weighting it by the number of item in each split, then substract from the current entropy. If the result is positive, we've lowered entropy with our split. The higher the result is, the more we've lowered entropy.

## Simple method is to use median to split values.

In [16]:
## The information gain for splitting on the age column of income.
import numpy as np

def calc_entropy(column):
    """
    Calculate entropy given a pandas series, list, or numpy array"""
    # Compute the counts of each unique value in the column
    counts = np.bincount(column)
    # Divdide by the total column length to get a probability
    probabilities = counts / len(column)
    
    # Initialize the entropy to 0
    entropy = 0
    # Loop through the probabilities, and add each one to the total entropy
    for prob in probabilities:
        if prob > 0:
            entropy += prob * math.log(prob, 2)
    return -entropy

In [17]:
entropy = calc_entropy([1,1,0,0,1])
print(entropy)

0.9709505944546686


In [18]:
information_gain = entropy - ((.8 * calc_entropy([1,1,0,0])) + (.2 * calc_entropy([1])))
print(information_gain)

0.17095059445466854


In [27]:
age_median = income['age'].median()
left_branch = income[income['age'] <= age_median]
right_branch = income[income['age'] > age_median]
left_prob = left_branch.shape[0] / income.shape[0]
right_prob = right_branch.shape[0] / income.shape[0]
entropy_high_income = calc_entropy(income['high_income'])
age_information_gain = entropy_high_income - ((left_prob * calc_entropy(left_branch['high_income'])) + 
                                             (right_prob * calc_entropy(right_branch['high_income'])))
age_information_gain

0.047028661304691965

In [28]:
calc_entropy(income['high_income'])

0.7963839552022132

## Find the variable to split on by calculating which split would have the highest information gain.

In [32]:
def calc_information_gain(data, split_name, target_name):
    """
    Calculate information gain given a data set, column to split on, and target
    """
    original_entropy = calc_entropy(data[target_name])
    
    # Find the median of the column for splitting
    column = data[split_name]
    median = column.median()
    
    # Make two subsets of the data, based on median
    left_split = data[column <= median]
    right_split = data[column > median]
    
    # Loop through the splits and calculate the subset entropies
    to_substract = 0
    for subset in [left_split, right_split]:
        prob = (subset.shape[0] / data.shape[0])
        to_substract += prob * calc_entropy(subset[target_name])
        
    # Return information gain
    return original_entropy - to_substract

print(calc_information_gain(income, "age", "high_income"))

0.047028661304691965


In [35]:
columns = ['age', 'workclass', 'education_num', 'marital_status', 'occupation', 'relationship', 
          'race', 'sex', 'hours_per_week', 'native_country']
information_gains = [calc_information_gain(income, column, 'high_income') for column in columns]

In [40]:
highest_gain = columns[information_gains.index(max(information_gains))]
highest_gain

'marital_status'

In [42]:
print(information_gains)

[0.047028661304691965, 0.006810984054396618, 0.06501298413277423, 0.1114272573715438, 0.0015822303843424645, 0.04736241665026941, 0.0, 0.0, 0.04062246867123487, 0.00013457344495848567]


# Part 2: Building a Decision Tree

## Recursion 